In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/My Drive/Colab Notebooks/recomendationSystem

/content/drive/My Drive/Colab Notebooks/recomendationSystem


In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
import sklearn

In [ ]:
df_solicitantes = pd.read_csv('solicitantes2020.csv')
df_solicitante_pedido = pd.read_csv('solicitante_pedido2020.csv')
print(df_solicitantes.shape)
print(df_solicitante_pedido.shape)

(43813, 10)
(38, 2)


Pessoa Física      41965
Pessoa Juridica     1848
Name: TipoDemandante

df_solicitantes.TipoDemandante.value_counts()

In [ ]:
dummies = pd.get_dummies(df_solicitantes['TipoDemandante'], drop_first = True)
df_solicitantes["TipoDemandante-Encoded"] = dummies

EMPTY        14776
Masculino    14734
Feminino     14069
Outros         234

In [ ]:
encoder = OrdinalEncoder(categories=[["Masculino", "Feminino", "Outros", " "]])
encoder.fit(df_solicitantes[["Genero"]])
df_solicitantes["Genero-Encoded"] = encoder.transform(df_solicitantes[["Genero"]])

df_solicitantes[["Genero", "Genero-Encoded"]].head(10)

In [ ]:
encoder = OrdinalEncoder(categories=[["SÃO PAULO", "RIO DE JANEIRO", "MINAS GERAIS", "DISTRITO FEDERAL", "RIO GRANDE DO SUL", "PARANÁ", "BAHIA", "SANTA CATARINA"
                                      , "PERNAMBUCO", "GOIÁS", "CEARÁ", "PARÁ", "ESPÍRITO SANTO", "PARAÍBA", "MATO GROSSO", "MARANHÃO"
                                      , "RIO GRANDE DO NORTE", "AMAZONAS", "MATO GROSSO DO SUL", "ALAGOAS", "PIAUÍ", "RONDÔNIA", "RONDÔNIA", "SERGIPE"
                                      , "TOCANTINS", "AMAPÁ", "RORAIMA", "ACRE", " "]])
encoder.fit(df_solicitantes[["UF"]])
df_solicitantes["UF-Encoded"] = encoder.transform(df_solicitantes[["UF"]])

In [ ]:
Lista_Municipio = df_solicitantes.Municipio.value_counts()
Lista_Municipio_Index = Lista_Municipio.index

encoder = OrdinalEncoder(categories=[Lista_Municipio_Index])
encoder.fit(df_solicitantes[["Municipio"]])
df_solicitantes["Municipio-Encoded"] = encoder.transform(df_solicitantes[["Municipio"]])

In [ ]:
Lista_Escolaridade = df_solicitantes.Escolaridade.value_counts()
Lista_Escolaridade_Index = Lista_Escolaridade.index

encoder = OrdinalEncoder(categories=[Lista_Escolaridade_Index])
encoder.fit(df_solicitantes[["Escolaridade"]])
df_solicitantes["Escolaridade-Encoded"] = encoder.transform(df_solicitantes[["Escolaridade"]])

In [ ]:
Lista_Profissao = df_solicitantes.Profissao.value_counts()
Lista_Profissao_Index = Lista_Profissao.index

encoder = OrdinalEncoder(categories=[Lista_Profissao_Index])
encoder.fit(df_solicitantes[["Profissao"]])
df_solicitantes["Profissao-Encoded"] = encoder.transform(df_solicitantes[["Profissao"]])

In [ ]:
Lista_Pais = df_solicitantes.Pais.value_counts()
Lista_Pais_Index = Lista_Pais.index

encoder = OrdinalEncoder(categories=[Lista_Pais_Index])
encoder.fit(df_solicitantes[["Pais"]])
df_solicitantes["Pais-Encoded"] = encoder.transform(df_solicitantes[["Pais"]])

In [ ]:
lista_solicitantes = df_solicitante_pedido['IDSolicitante'].values
df_solicitantes = df_solicitantes.query('IdSolicitante in @lista_solicitantes')

In [ ]:
index = df_solicitantes[['IdSolicitante']]
headers = ['IdSolicitante']
df_solicitantes_index = pd.DataFrame(index.values, columns=headers)

data = df_solicitantes[['TipoDemandante-Encoded','Genero-Encoded','UF-Encoded','Municipio-Encoded','Escolaridade-Encoded','Profissao-Encoded','Pais-Encoded']]
headers = ['TipoDemandante','Genero','UF','Municipio','Escolaridade','Profissao','Pais']
df_solicitantes_vectores = pd.DataFrame(data.values, columns=headers)

In [ ]:
df_solicitantes_index.head(5)

,IdSolicitante
0,97333
1,225608
2,394628
3,1142425
4,1148742


In [ ]:
df_solicitantes_vectores.head(5)

,TipoDemandante,Genero,UF,Municipio,Escolaridade,Profissao,Pais
0,0.0,0.0,2.0,1330.0,0.0,0.0,1.0
1,0.0,0.0,3.0,2.0,0.0,0.0,1.0
2,0.0,0.0,26.0,52.0,0.0,0.0,0.0
3,0.0,3.0,28.0,0.0,0.0,0.0,1.0
4,0.0,3.0,28.0,0.0,0.0,0.0,0.0


In [ ]:
df_solicitante_pedido.head(5)

,IDSolicitante,IDPedido
0,97333,1812392
1,176782,1840724
2,225608,1983287
3,394628,1625700
4,1142425,1628891


In [ ]:
lista_id_solicitantes = [x for x in df_solicitante_pedido['IDSolicitante']]
y = df_solicitantes_index.values

In [ ]:
def valores_proximos(id_solicitantes):  
  return np.linalg.norm(df_solicitantes_vectores - id_solicitantes, axis=1)

In [ ]:
def knnSolicitantes(distances,k):
  return distances.argsort()[:k]

In [ ]:
K = 5
id_solicitantes_knn = {}

for id_solicitante in lista_id_solicitantes:
  distances = valores_proximos(id_solicitante)
  nearest_neighbor_ids = knnSolicitantes(distances,K)
  list_nn_ids = [x.item() for x in y[nearest_neighbor_ids]]
  id_solicitantes_knn[id_solicitante] = list_nn_ids

In [ ]:
id_solicitantes_knn
df = pd.Series(id_solicitantes_knn, name='IDSimilitudes').rename_axis('IDSolicitante').explode().reset_index()
df.head(10)

,IDSolicitante,IDSimilitudes
0,97333,97333
1,97333,3077863
2,97333,2933509
3,97333,3117987
4,97333,394628
5,176782,97333
6,176782,3077863
7,176782,2933509
8,176782,3117987
9,176782,394628


In [ ]:
df[df['IDSolicitante'] == 3172106]

,IDSolicitante,IDSimilitudes
140,3172106,97333
141,3172106,3077863
142,3172106,2933509
143,3172106,3117987
144,3172106,394628


In [ ]:
df_solicitantes_index[df_solicitantes_index['IdSolicitante'] == 3172106]
# 3127547

,IdSolicitante
20,3172106
